<a href="https://colab.research.google.com/github/ttogle918/news_by_kobert/blob/master/%EB%8B%A8%EB%9D%BD%EC%97%B0%EA%B2%B0%EC%97%AC%EB%B6%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#3주차 : 단락 연결 여부 확인

## 1. 단락 유사도 파악

2개 단락의 유사도 또는 연결성을 확인하는 것이 목적이다.  
이를 위해 KLUE(Korean Lnaguage Understanding Evaluation)를 통해 배포된 BERT 모델을 이용할 것이다. 

In [74]:
!pip install transformers

In [75]:
from transformers import TFBertForMaskedLM, TFBertForNextSentencePrediction, AutoTokenizer, FillMaskPipeline, BertTokenizer, PreTrainedTokenizer, BertConfig
import tensorflow as tf
import torch
from tqdm import tqdm
import json
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
from datetime import datetime
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

####  KLUE는 총 8가지 자연어이해(NLU) task로 구성되어 있다.  

    -Topic Classification  
    -Semantic Textual  
    -Natural Language Inference  
    -Named Entity Recognition  
    -Relation Extraction  
    -Dependency parsing  
    -Machine Reading Comprehension  
    -Dialogue State Tracking  

이 중에서 여기서 진행할 것은 Natural Language Inference(자연어추론)이다.  


## 방법1. MASK에 들어갈 단어 예측

In [76]:
model = TFBertForMaskedLM.from_pretrained('klue/bert-base', from_pt=True)  #pytorch로 학습된 모델을 가져와 tf에서 사용
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")   #해당 모델이 학습되었을 당시 사용된 토크나이저

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [77]:
inputs = tokenizer('축구는 정말 재미있는 [MASK]다.', return_tensors='tf')  
print(inputs['input_ids'])
print(inputs['token_type_ids'])   #문장 길이만큼의 0 시퀀스
print(inputs['attention_mask'])   #실제 단어와 패딩 토큰을 구분하기 위한 어텐션 마스크

tf.Tensor([[   2 4713 2259 3944 6001 2259    4  809   18    3]], shape=(1, 10), dtype=int32)
tf.Tensor([[0 0 0 0 0 0 0 0 0 0]], shape=(1, 10), dtype=int32)
tf.Tensor([[1 1 1 1 1 1 1 1 1 1]], shape=(1, 10), dtype=int32)


In [78]:
pip = FillMaskPipeline(model=model, tokenizer=tokenizer)   #[MASK] 위치에 들어갈 상위 5개 출력
pip('축구는 정말 재미있는 [MASK]다.')

[{'score': 0.8963507413864136,
  'sequence': '축구는 정말 재미있는 스포츠 다.',
  'token': 4559,
  'token_str': '스포츠'},
 {'score': 0.025957776233553886,
  'sequence': '축구는 정말 재미있는 거 다.',
  'token': 568,
  'token_str': '거'},
 {'score': 0.010033952072262764,
  'sequence': '축구는 정말 재미있는 경기 다.',
  'token': 3682,
  'token_str': '경기'},
 {'score': 0.007924409583210945,
  'sequence': '축구는 정말 재미있는 축구 다.',
  'token': 4713,
  'token_str': '축구'},
 {'score': 0.00784420408308506,
  'sequence': '축구는 정말 재미있는 놀이 다.',
  'token': 5845,
  'token_str': '놀이'}]

In [79]:
#원래 단어는 '지지자'
pip('더불어민주당 [MASK] 중 39.5%, 국민의당 지지자 중 28.8%, 기타 21%, 무당층의 16.2%, 개혁보수당 지지자 중 12.3%였다.')   #원래 단어는 '지지자'

[{'score': 0.9840474724769592,
  'sequence': '더불어민주당 지지자 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.',
  'token': 11006,
  'token_str': '지지자'},
 {'score': 0.014456203207373619,
  'sequence': '더불어민주당 지지층 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.',
  'token': 13755,
  'token_str': '지지층'},
 {'score': 0.0006135492003522813,
  'sequence': '더불어민주당 응답자 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.',
  'token': 9440,
  'token_str': '응답자'},
 {'score': 0.00022629150771535933,
  'sequence': '더불어민주당 지지 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.',
  'token': 4315,
  'token_str': '지지'},
 {'score': 0.0002214877458754927,
  'sequence': '더불어민주당 유권자 중 39. 5 %, 국민의당 지지자 중 28. 8 %, 기타 21 %, 무당층의 16. 2 %, 개혁보수당 지지자 중 12. 3 % 였다.',
  'token': 7243,
  'token_str': '유권자'}]

In [80]:
#원래 단어는 '이런'
pip('75.3%는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1.5%였다. [MASK] 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.')

[{'score': 0.527259886264801,
  'sequence': '75. 3 % 는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 이 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.',
  'token': 1504,
  'token_str': '이'},
 {'score': 0.19859308004379272,
  'sequence': '75. 3 % 는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 조사 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.',
  'token': 3742,
  'token_str': '조사'},
 {'score': 0.17498789727687836,
  'sequence': '75. 3 % 는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 이런 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.',
  'token': 3667,
  'token_str': '이런'},
 {'score': 0.043650247156620026,
  'sequence': '75. 3 % 는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1. 5 % 였다. 이번 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.',
  'token': 3686,
  'token_str': '이번'},
 {'score': 0.023734353482723236,
  'sequence': '75. 3 % 는 전혀 참여한 적이

## 다음 문장 예측

In [81]:
model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForNextSentencePrediction: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForNextSentencePrediction from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForNextSentencePrediction were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForNextSentencePrediction for predictions without further training.


In [82]:
# XML파일에서 추출한 이어지는 두 개의 문장
prompt = '이런 결과를 지난해 11월 행정자치부가 발표한 우리나라 인구수 5168만여명에 대입하면 ' 
next_sentence = ' 촛불집회 참가 경험이 있는 국민은 1199만여명으로 나온다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [83]:
# XML파일에서 추출한 이어지는 제목, 문장
prompt = '서브프라임 모기지 사태로 촉발된 미국의 금융위기가 미국 금융기관을' 
next_sentence = ' 몰락시키고, 세계 굴지의 자동차 회사들을 생존위기로 내몬 데 이어 최근 언론사까지 벼랑 끝으로 내몰고 있다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [0]


In [84]:
# XML파일에서 추출한 상관없는 두 개의 문장
prompt = '75.3%는 전혀 참여한 적이 없다고 응답했고, 모른다고 답하거나 응답하지 않은 비율은 1.5%였다.' 
next_sentence = '이 목록에 오른 493개 차량 모델 중 LG화학과 삼성SDI의 배터리를 탑재한 차량은 하나도 없었다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


In [85]:
# XML파일에서 추출한 상관없는 제목, 문장
prompt = '남의 집 제집이라 속여 팔았는데 사기 아니라고?' 
next_sentence = '충남 금산군이 환경부가 수변구역으로 지정한 임야에 버섯재배사 개발허가를 내줘 빈축을 사고 있다.'
encoding = tokenizer(prompt, next_sentence, return_tensors='tf')

logits = model(encoding['input_ids'], token_type_ids=encoding['token_type_ids'])[0]

softmax = tf.keras.layers.Softmax()
probs = softmax(logits)
print('최종 예측 레이블 :', tf.math.argmax(probs, axis=-1).numpy())

최종 예측 레이블 : [1]


## model = TFBertForNextSentencePrediction.from_pretrained('klue/bert-base', from_pt=True) 에
## 모두의말뭉치 Corpus를 이어서 학습

In [86]:
#모두의말뭉치 파일 내 json 파일 목록
def parse_paths(folder):
    for current, dirs, files in os.walk(folder):
        return [os.path.join(current, file) for file in files if file.endswith(".json")]

#json 파일 내 기사 내용 추출
def parse_contents(files):
    result = []
    for file in tqdm(files, desc="[Contents Parsing]"):    #진행상황 확인용
        with open(file, "r", encoding="utf-8") as f:
            contents = json.load(f)
            for doc in contents['document']:
                result_paragraph = []     #같은 기사 한문단
                paragraph_id = doc['id']  #같은 기사 id
                for paragraph in doc['paragraph']:
                    if paragraph['id'][-2:] == '.1':   #기사의 제목 부분은 제외
                        continue
                    elif paragraph['id'][:17]==paragraph_id[:17] and paragraph['id'][-2:]=='.2':  #같은 id(같은 단락의 기사)의 첫줄이라면
                        continue
                    elif paragraph['id'][:17]==paragraph_id[:17] and paragraph['id'][-2:]!='.2':  #같은 id(같은 단락의 기사)의 첫줄이 아닌 문장
                        result_paragraph.append(paragraph['form'])
                    else:   #다른 id(다른 단락의 기사)라면
                        result.append(result_paragraph)   #지금까지 id의 문장들은 result로
                        result_pragraph = []   #result_paragraph 비우기
                        paragraph_id = paragraph['id'][:17]
                        result_paragraph.append(paragraph['form'])
    print("[Contents Length] {0:,}".format(len(result)))
    
    for i in range(len(result)):
        pa = ''
        for sentence in result[i]:
            pa += f' {sentence}'
            result[i] = pa    
    return result

data = parse_paths('/content/drive/MyDrive/Korpus')
contents = parse_contents(data)

[Contents Parsing]: 100%|██████████| 35/35 [01:32<00:00,  2.65s/it]

[Contents Length] 314


In [87]:
#같은 기사는 한 문단으로 처리됨
contents[0]

' 전남 목포시는 최근 이목이 집중되고 있는 근대역사문화공간 재생활성화 사업을 근대문화재 보존과 활용이라는 당초 취지대로 차질없이 추진하겠다고 밝혔다. 이 사업은 목포 원도심인 유달‧만호동 일대에 산재해 있는 근대건축물 등 문화유산을 보존하고 보수‧정비하는 사업으로 금년부터 향후 5년간 총 사업비 500억원이 투입된다. 목포시는 원도심 일대의 근대경관을 회복하고 거주민 생활여건 개선과 관광인프라 확충 등을 통해 이 지역을 전국적인 근대 문화유산의 보고로 만들어 나간다는 계획이다. 아울러, 최근 언론의 집중 보도로 목포 근대문화재에 대해 관심이 많아진 이 기회를 문화유산 보존의 필요성과 당위성을 널리 알리고 이에 대한 사회적 합의를 이루는 계기로 만드는 노력도 기울이기로 했다. 올 해는 종합정비계획을 수립하고 역사문화공간 내 건축자산 매입 및 정비에 나선다. 개별문화재로 등록된 15개소를 중심으로 우선 매입하고, 역사적‧건축적 가치가 높은 건축물과 경관을 훼손하는 건축물도 매입해서 공공재로의 활용을 확대한다는 계획이다. 특히, 목포시는 건축자산 매입 시 공정하고 투명한 행정절차를 통해 투기자본 유입을 원천 차단한다. 또, 근대역사문화공간 내 보존 활용, 관리 및 지원 기준에 관한 조례를 제정해 젠트리피케이션 발생을 예방하고, 특정 투기세력들이 수혜를 받을 수 없도록 제도적 장치를 마련하기로 했다. 목포시는 근대역사문화공간 재생 활성화 시범사업이 목포의 역사적 가치를 보존하면서 지역발전도 함께 이룰 수 있는 중요한 기회이자, 소중한 문화유산을 지키기 위해 꼭 필요한 사업이라는 점을 강조했다. 아울러, 문화재청, 관련 기관 등과 잘 협력해 사업을 본래의 취지대로 흔들림없이 추진해서 반드시 근대문화재 보존활용의 성공 모델로 만들어 간다는 방침을 확고히 했다.'

In [88]:
len_paragraph = []
for i in contents:
    len_paragraph.append(len(i))
    
print(f'기사의 길이 : 최소 {min(len_paragraph)}자, 최대 길이 {max(len_paragraph)}자')
#256자로 나누어 pre-trained 모델에 fine-tuning하기 위한 입력값으로 사용
#최대길이인 4135자를 입력값으로 쓴다면, 4135 // 256 = 16개의 문단으로 나누어
#각 문단의 다음 문단을 label로 입력

기사의 길이 : 최소 406자, 최대 길이 4135자


## 마무리
맞춤법 검사 : py-hanspell